In [1]:
print("Imports")
from keras.models import load_model
from Class_Image import Image
from os.path import abspath
from Class_training import *
from Model_comparisons import *
from os.path import abspath, join
import numpy as np

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import VGG19
#import keras
from tensorflow.keras import Input, Model
from keras.preprocessing.image import ImageDataGenerator
import keras_tuner as kt

%matplotlib inline

Imports


In [2]:
def map_fn(serialized_example):
    feature = {
        'data_label': tf.io.FixedLenFeature([2], tf.int64),
        'data_right': tf.io.FixedLenFeature([150528], tf.int64),
        'data_left': tf.io.FixedLenFeature([150528], tf.int64),
    }
    
    ex = tf.io.parse_single_example(serialized_example, feature)
    return ex, ex['data_label']

In [3]:
#conv_model = comparisons_model(224)

tuner = kt.Hyperband(comparisons_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='Z:/tuner',
                     project_name='hp_layer_freeze')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


INFO:tensorflow:Reloading Oracle from existing project Z:/tuner\hp_layer_freeze\oracle.json
INFO:tensorflow:Reloading Tuner from Z:/tuner\hp_layer_freeze\tuner0.json


In [4]:
dataset_val = tf.data.TFRecordDataset((
    'Z:/Comparison_1_npy/data_val.tfrecord'
)
)
dataset_train = tf.data.TFRecordDataset((
    'Z:/Comparison_1_npy/data_train.tfrecord', 
    #'Z:/data_train_synthetic.tfrecord'
)
)
dataset_train = dataset_train.map(map_fn)
dataset_train = dataset_train.shuffle(2048, reshuffle_each_iteration = True)
dataset_train = dataset_train.prefetch(buffer_size=tf.data.AUTOTUNE)
dataset_train = dataset_train.batch(6)

dataset_val = dataset_val.map(map_fn)
#dataset_val = dataset_val.shuffle(2048)
dataset_val = dataset_val.prefetch(buffer_size=tf.data.AUTOTUNE)
dataset_val = dataset_val.batch(6)

In [ ]:
#history1=conv_model.fit(dataset_train, validation_data = dataset_val, epochs = 100)
tuner.search(dataset_train, validation_data = dataset_val, epochs=500, callbacks=[stop_early], verbose=2)

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
layers_frozen     |2                 |20                
translate         |0.3               |0                 
zoom              |0                 |0.1               
rotation          |0.1               |0.7               
dropout           |0.4               |0.2               
learning_rate     |9.2569e-06        |2.4351e-05        
tuner/epochs      |10                |10                
tuner/initial_e...|0                 |4                 
tuner/bracket     |0                 |2                 
tuner/round       |0                 |2                 

Epoch 1/10


c:\users\lmacl\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\functional.py:588: UserWarning: Input dict contained keys ['data_label'] which did not match any model input. They will be ignored by the model.
  warnings.warn(
